<a href="https://colab.research.google.com/github/Nilufayeasmin299/Reproduce-GNN_Ownership_Verification/blob/main/GNN_Ownership_using_PubMed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install Required Libraries
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install torch_geometric
!pip install numpy scikit-learn tqdm pyyaml argparse


Looking in indexes: https://download.pytorch.org/whl/cu118
  Using cached argparse-1.4.0-py2.py3-none-any.whl.metadata (2.8 kB)
Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [ ]:
# Step 2: Import Libraries and Set Configuration
import torch
import numpy as np

# Configuration
config = {
    'dataset': 'PubMed',  # Updated dataset to PubMed
    'target_model': 'gat',  # GNN architecture: gcn, gat, sage, GIN, SGC
    'target_hidden_dims': [352, 256],
    'embedding_dim': 128,
    'epochs': 100,
    'learning_rate': 0.01,
    'mask_ratio': 0.2,  # Ratio of features to mask
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'n_runs': 3,  # Number of runs for averaging metrics
}

# Set Random Seeds for Reproducibility
torch.manual_seed(42)
np.random.seed(42)
if config['device'] == 'cuda':
    torch.cuda.manual_seed_all(42)

In [ ]:
# Step 3: Define Feature Masking Function
def mask_features(data, mask_ratio):
    """Apply feature masking to enrich the model's fingerprint."""
    num_features = data.x.shape[1]
    mask = np.random.choice([0, 1], size=num_features, p=[mask_ratio, 1 - mask_ratio])
    data.x = data.x * torch.tensor(mask, dtype=torch.float32, device=config['device'])
    return data

In [ ]:
# Step 4: Define GNN Model
from torch_geometric.nn import GCNConv, GATConv

class GNNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dims, output_dim, model_type='gcn'):
        super(GNNModel, self).__init__()
        if model_type == 'gcn':
            self.conv1 = GCNConv(input_dim, hidden_dims[0])
            self.conv2 = GCNConv(hidden_dims[0], hidden_dims[1])
        elif model_type == 'gat':
            self.conv1 = GATConv(input_dim, hidden_dims[0])
            self.conv2 = GATConv(hidden_dims[0], hidden_dims[1])
        else:
            raise ValueError(f"Unsupported model type: {model_type}")
        self.fc = torch.nn.Linear(hidden_dims[1], output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = torch.relu(self.conv1(x, edge_index))
        x = torch.relu(self.conv2(x, edge_index))
        x = self.fc(x)
        return torch.log_softmax(x, dim=1)


In [ ]:
# Step 5: Train and Test Functions
from sklearn.metrics import accuracy_score

def train_model(model, data, optimizer, criterion, epochs):
    """Train the GNN model."""
    model.train()
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()

def test_model(model, data):
    """Evaluate the model's accuracy on the test set."""
    model.eval()
    out = model(data)
    pred = out.argmax(dim=1)
    acc = accuracy_score(data.y[data.test_mask].cpu(), pred[data.test_mask].cpu())
    return acc

In [ ]:
# Step 6: Evaluate Ownership Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

def evaluate_classifier(target_model, shadow_model, data):
    """Train and evaluate the ownership verification classifier."""
    target_model.eval()
    shadow_model.eval()

    target_embeddings = target_model(data).detach().cpu().numpy()
    shadow_embeddings = shadow_model(data).detach().cpu().numpy()

    X = np.vstack([target_embeddings, shadow_embeddings])
    y = np.array([1] * len(target_embeddings) + [0] * len(shadow_embeddings))  # 1 = Extracted, 0 = Independent

    # Split data for training and testing
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train ownership classifier
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)

    # Evaluate classifier
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    # Confusion Matrix
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

    return acc, fpr, fnr


In [ ]:
# Step 7: Load Dataset
from torch_geometric.datasets import Planetoid

# Load PubMed Dataset
dataset = Planetoid(root='./data', name=config['dataset'])
data = dataset[0].to(config['device'])

# Apply Feature Masking
data = mask_features(data, config['mask_ratio'])

Processing...
Done!


In [ ]:
# Step 8: Initialize Models
target_model = GNNModel(dataset.num_features, config['target_hidden_dims'], dataset.num_classes, config['target_model']).to(config['device'])
shadow_model = GNNModel(dataset.num_features, config['target_hidden_dims'], dataset.num_classes, config['target_model']).to(config['device'])


In [ ]:
# Step 9: Train Models
optimizer = torch.optim.Adam(target_model.parameters(), lr=config['learning_rate'])
criterion = torch.nn.NLLLoss()

train_model(target_model, data, optimizer, criterion, config['epochs'])
train_model(shadow_model, data, optimizer, criterion, config['epochs'])

In [ ]:
 #Step 10: Evaluate Ownership Classifier
acc, fpr, fnr = evaluate_classifier(target_model, shadow_model, data)
print(f"Ownership Classifier Accuracy: {acc:.4f}, FPR: {fpr:.4f}, FNR: {fnr:.4f}")

Ownership Classifier Accuracy: 1.0000, FPR: 0.0000, FNR: 0.0000
